In [1]:
import sys
sys.path.append("libs/")
from mysql import connector 
import os
initdir = os.getcwd() + '/'
srcdir = "./../Unix_Source/"
small = "unix_small_2/"
large = "unix_cmp/"
os.chdir('libs/')
from sql_connection import mysql_connection,true_order
from make import make_dir,make_prg,opts,ch_opt_string,opts_lvl
import pickle
os.chdir(initdir)
import subprocess
from functools import reduce
from tqdm import tqdm
order = pickle.load(open('./libs/order.dmp','rb'))
traversal = pickle.load(open('./libs/trv.dmp','rb'))
true_order = pickle.load(open('./libs/true_order.dmp','rb'))



# Check existance size difference

In [7]:
mysql_sizes = {}
D
cursor = conn.cursor()
os.chdir(initdir+srcdir+small)
for file in os.listdir():
    if os.path.isdir(file):
        mysql_sizes[file] = {}
        cursor.execute("SELECT ID FROM PG WHERE PG_NAME='%s'" %file)
        pg_id = cursor.fetchone()[0]
        cursor.execute("SELECT ID FROM TARGET WHERE PG_ID=%s" %pg_id)
        trg = cursor.fetchone()[0]
        cursor.execute("SELECT LL_ID,LL_NAME FROM RL_LL WHERE TARGET_ID=%s" %trg) 
        ll_id,ll_name = cursor.fetchone()
        cursor.execute("SELECT DATA_ID,OPT_LVL_ID FROM MAIN_QUERY WHERE RL_LL_ID=%s" %ll_id)
        for data_id,opt_lvl_id in cursor.fetchall():
            cursor.execute("SELECT SIZE FROM TARGET_RES WHERE OPT_LVL_ID=%s AND TARGET_ID=%s" %(opt_lvl_id,trg))
            mysql_sizes[file][opt_lvl_id] = {'data':data_id,'size':cursor.fetchone()[0]}

In [56]:
most_res = []
for file in mysql_sizes:
    s = set(map(lambda x: mysql_sizes[file][x]['size'],mysql_sizes[file]))
    if len(s)>1:
        most_res.append((file,len(s)))
max_file = max(most_res,key=lambda x : x[1])[0]


## Create true order

In [3]:
os.chdir(initdir)
desc_opt_lvls = pickle.load(open('./libs/desc_opt_lvls.dmp','rb'))
true_order = {}
for lvl_opt in order:
    true_order[lvl_opt] = {'order':[],'init':desc_opt_lvls[lvl_opt]['start']}
    min_ = 1
    for entry in traversal[lvl_opt]:
        if min_< traversal[lvl_opt][entry][1]:
            min_ = traversal[lvl_opt][entry][1]
    used = []
    for ind in range(1,min_):
        used.append(order[lvl_opt][ind]['pass'])
    true_order[lvl_opt]['used'] = used
    for ind in range(min_,len(order[lvl_opt])):
        true_order[lvl_opt]['order'].append(order[lvl_opt][ind]['pass'])
#     print(len(true_order[lvl_opt]['used']),len(true_order[lvl_opt]['order']))           

In [66]:
test_res = {}
os.chdir(initdir+srcdir+small)
mk_dir = make_dir("{}/{}/".format(max_file,'source'))[0]
os.chdir(mk_dir)
for lvl_opt in tqdm(opts_lvl):
    used = true_order[lvl_opt]['used']
    order_ = []
    test_res[lvl_opt] = {}
    for option in true_order[lvl_opt]['order']:
        for opt_lvl in opts_lvl
        order_.append(option)
        ch_opt_string(used+order_,initdir+srcdir+small+mk_dir,true_order[opt_lvl]['init'].split(' '))
        make_prg(['make'])
        res =int(subprocess.check_output("wc -c {}".format('lcd'), stderr=subprocess.STDOUT,shell=True).decode('utf-8')[:-1].split(' ')[0])
        test_res[lvl_opt][option] = res
        make_prg(['make','clean'])


100%|██████████| 5/5 [03:00<00:00, 36.16s/it]


In [92]:
conn = mysql_connection()
cursor = conn.cursor()
for opt_lvl in test_res:
    for option in test_res[opt_lvl]:
        cursor.execute("SELECT ID FROM OPTS_LVL WHERE NAME='%s' AND OPT_ID=(SELECT ID FROM OPTIONS WHERE NAME='%s')" %(opt_lvl,option))
        opt_id = cursor.fetchone()[0]
        if int(test_res[opt_lvl][option]) != int(mysql_sizes[max_file][opt_id]['size']):
            print("OPTION_LVL:{} OPTION:{} DIFF_SIZE:{}".format(opt_lvl,option,str(int(mysql_sizes[max_file][(opt_id)]['size'])-int(test_res[opt_lvl][option]))))
            

OPTION_LVL:opt_O2 OPTION:-inline DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-functionattrs DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-sroa DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-memoryssa DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-early-cse-memssa DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-speculative-execution DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-lazy-value-info DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-jump-threading DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-correlated-propagation DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-libcalls-shrinkwrap DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-branch-prob DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-block-freq DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-pgo-memop-opt DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-tailcallelim DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-reassociate DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-loop-simplify DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-lcssa-verification DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-lcssa DIFF_SIZE:-32
OPTION_LVL:opt_O2 OPTION:-scal